# Encoding a neural network generated with Julia into a JSON file (regression)

- Step 1: Generate a regression dataset
- Step 2: Write the dataset to a CSV file
- Step 3: Initialize the model
- Step 4: Train the neural network
- Step 5: Write the parameters of the trained neural network to a JSON file
- Step 6: Laplace Approximation and show hessian

In [1]:
using Pkg; Pkg.activate(".")
# Import libraries
using Flux, Plots, Random, Statistics, LaplaceRedux, MLUtils, JSON, Serialization

  Activating

 new project at `c:\Users\adeli\OneDrive\Desktop\facultate\2nd year\Q4 - Software Project\LaplaceRedux.jl\dev\notebooks\nn_encoding`


### Step 1: Generate a regression dataset

In [2]:
using LaplaceRedux.Data
n = 300       # number of observations
σtrue = 0.30  # true observational noise
x, y = Data.toy_data_regression(n; noise=σtrue)
xs = [[x] for x in x]
X = permutedims(x)

data = zip(xs, y)

print(data)

zip([[7.233209025552167], [5.370305321628792], [1.3204445174222412], [3.5257809240521976], [5.582714669720058], [2.6722865671239084], [2.732145420667517], [1.203226084467075], [2.28743936115257], [0.958008612863134], [5.994395020285243], [7.73748221798943], [1.029343314435236], [6.245407649870837], [0.8709327812045373], [5.310163933774123], [6.647500492261089], [5.082744036461372], [2.605494280144839], [6.468929888525023], [5.931883059417159], [7.192414707135119], [6.842481001474571], [3.7694257426204683], [0.13104025932725083], [1.253433079033715], [7.425096275638605], [3.9679415896752808], [6.462524808602669], [7.832925496832232], [1.6791735992943009], [4.748200019226438], [2.6927255891728326], [7.750121537458007], [1.3936421850822986], [4.942546289635383], [6.213793782316175], [2.616426999840429], [2.364565045867767], [5.582195961446241], [6.722357380002154], [1.7291640870914966], [1.6105905665052154], [4.398687706419872], [6.890973981792012], [2.5251673908678756], [6.07065328970647

-1.1009968501772256, 0.6645163784856851, 0.517403580911474, 0.3201952521152393, 0.8108330007393733, 1.232160186828724, 0.7943442117694917, 0.9062230562673812, 0.6822930152555764, -0.13451095386637235, 0.508480504327735, 0.31293559315093056, -1.3458441550935547, -0.6088939666811473, -0.3442709571178091, 0.46223342843545934, -0.8850096029342049, -0.9676112362686763, 0.5766420945812285, 0.4842680316471556, -0.891435885594591, 0.7282110018494258, 0.34683338200111813, 0.5453401930215733, 0.08002674769137291, 0.8205037864157045, 1.358817861600715, 0.8852762059426896, 0.5757091889675138, 0.990644136695064, -0.09282871831900297, 0.5327174288730347, 0.17110102264902383, 0.9316636908894509, 0.8663041834225641, 1.2240904706545537, 0.12299154929501183, 0.6562609687443921, -1.2634317471480494, -0.8893451046296732, 0.9515449235668691, -0.7648422016725138, 0.13553707836428663, -0.12930822950864462, 0.4699331239173961, -1.371411020307445, 0.9113039833304852, 0.9321451737411656, 0.7423442880785638, 0.7

 -0.47622534385473064, 0.4862494846491263, -1.136286403452374, -0.47262826498246613, 1.013427685326901, -0.015737420745560837, 1.2449877748144527, 0.7828247819760394, -0.8626331554749682, 1.2307781155129718, -0.02004780130126859, -0.5230606414106841, 0.4165142807814214, -1.1820124906702643, 1.2710155183461018, 0.6242269224904773, 0.6711419641482447, -0.34330025662409547, -1.0141326250412441, 0.933239781565363, 1.420932615258128, -0.8262482393877293, -0.10915312729859977, 0.15636602852724613, -0.8050191864215389, -0.028901156028514216, 0.41135766469050455, -0.9055381524620365, 0.50313665843093, -1.2568464269601705, 0.3014195260914135, -0.00636009367464728, 0.652415562687717])

### Step 2: Write the dataset to a CSV file

In [3]:
file = "data_regression.csv"
csv_file = open(file, "w")

# Write the header
write(csv_file, "xs,y\n")

# Write the data
for (xs, y) in zip(x, y)
    write(csv_file, "$xs,$y\n")
end

# Close the CSV file
close(csv_file)

### Step 3: Initialize the model

In [4]:

n_hidden = 10
D = size(X, 1)
nn = Chain(
    Dense(D, n_hidden, tanh),
    Dense(n_hidden, 1)
)  
loss(x, y) = Flux.Losses.mse(nn(x), y)

loss (generic function with 1 method)

### Step 4: Train the neural network

In [5]:
using Flux.Optimise: update!, Adam
opt = Adam(1e-3)
epochs = 1000
avg_loss(data) = mean(map(d -> loss(d[1], d[2]), data))
show_every = epochs/10

for epoch = 1:epochs
  for d in data
    gs = gradient(Flux.params(nn)) do
      l = loss(d...)
    end
    update!(opt, Flux.params(nn), gs)
  end
  if epoch % show_every == 0
    println("Epoch " * string(epoch))
    @show avg_loss(data)
  end
end

┌ Warning: Layer with Float32 parameters got Float64 input.
│   The input will be converted, but any earlier layers may be very slow.
│   layer = Dense(1 => 10, tanh)
│   summary(x) = 1-element Vector{Float64}
└ @ Flux C:\Users\adeli\.julia\packages\Flux\FWgS0\src\layers\stateless.jl:50


Epoch 100


avg_loss(data) = 0.09011571400513423


Epoch 200
avg_loss(data) = 0.08708992553462319


Epoch 300
avg_loss(data) = 0.08666497402898633


...

Epoch 600
avg_loss(data) = 0.08599713323791576


Epoch 700
avg_loss(data) = 0.08582459610697407


Epoch 800
avg_loss(data) = 0.08566007118773253


Epoch 900
avg_loss(data) = 0.08550218137883826


Epoch 1000
avg_loss(data) = 0.08535026793796623


### Step 5: Write the parameters of the trained neural network to a JSON file

In [6]:
serialize_json_nn(nn::Chain)::String = JSON.json([Dict(:weight => nn.layers[i].weight, :bias => nn.layers[i].bias) for i in range(1, length(nn.layers))])
# Export as JSON
write("nn_regression.json", serialize_json_nn(nn))
serialize("nn-binary_regression.jlb", nn)

Flux.params(nn)

Params([Float32[-0.55855876; 0.2368329; … ; -0.24539436; 0.6470228;;], Float32[-0.42261407, -0.21227753, -0.0096151605, -0.7529069, 1.3016218, 3.116596, -2.1422381, -1.2806743, 0.78418183, -1.3360132], Float32[-0.27233723 0.7866149 … -1.1626163 -0.75578994], Float32[0.13576016]])

### Step 6: Laplace Approximation and show hessian

In [7]:
la = Laplace(nn; likelihood=:regression, hessian_structure=:full, subset_of_weights=:all, backend=:GGN)
fit!(la, data)
@show la.posterior.H

la.posterior.H = [1.0338194977674675 -14.892659733548498 -1.2742829297078053 -0.8511076819554622 27.444908382625727 -15.381136766982422 18.800298057968575 -27.042541434491795 24.21986570891022 11.642838771667357 0.9595230163745754 -8.778792184733902 -1.2438670861358099 -0.8763172200753753 11.975118542875862 -5.225828699773501 8.229480028534795 -11.865532335388707 12.15581434838532 6.4046639894891655 10.382391007675324 -2.6061978136131074 -9.326744543351197 -7.750272786128335 -7.496409565559588 -9.41925737290876 5.750626389286481 7.380455697675643 -3.6600664781872183 1.6077879471704364 -12.426787402946502; -14.892659733548498 604.6819151572854 15.372755011605477 7.791888649311915 -2176.257645118909 982.9237094649375 -670.5824288322829 2119.1036397000353 -1408.5810990462196 -310.0439222608329 -8.77879218194721 188.26422267593443 10.305700369264741 6.385025862359035 -514.9938981896266 263.7421574793261 -226.37670829793205 503.49568574130535 -374.03548060311005 -120.30204302957281 -380.464

31×31 Matrix{Float64}:
   1.03382     -14.8927    -1.27428   -0.851108  …      1.60779    -12.4268
 -14.8927      604.682     15.3728     7.79189        233.098      398.268
  -1.27428      15.3728     1.60966    1.11794         -3.64847     14.0326
  -0.851108      7.79189    1.11794    0.832402        -3.67101      8.35266
  27.4449    -2176.26     -23.9343    -9.279        -1206.62     -1467.92
 -15.3811      982.924     12.1658     3.04485   …    415.298      552.542
  18.8003     -670.582    -17.9554    -6.76299       -193.134     -386.96
 -27.0425     2119.1       23.6686     9.22925       1170.01      1428.29
  24.2199    -1408.58     -23.0007   -10.2078        -684.038     -932.679
  11.6428     -310.044    -12.3364    -6.04432        -62.4711    -192.36
   ⋮                                             ⋱                   ⋮
  -9.32674     374.585      9.95729    5.4459         154.517      261.702
  -7.75027     368.483      7.79846    3.83005        171.07       241.941
  -7.4